In [1]:
import json
import math
import os
from datetime import timedelta  

import numpy as np
import pandas as pd
import PIL
from PIL import Image

### Load Data

In [2]:
radar_folder = '/data1/scidata/meteotn_data_new'
data_folder = './public'
os.makedirs(data_folder, exist_ok=True)

In [3]:
img_size  = 64

In [4]:
# train and val runs are normalized between 0 and 1
images = os.path.join(radar_folder, 'train_data.npz')
# images = os.path.join(radar_folder, 'val_data.npz')

images = np.load(images)['arr_0']
images = (images - images.min()) / (images.max() - images.min())
images = np.round(images * 255).astype(np.uint8)
images.shape

(200000, 64, 64)

In [5]:
images = images[150000:200000]

In [6]:
images.shape

(50000, 64, 64)

In [7]:
ss_width = 2048

In [8]:
image_height = img_size

In [9]:
image_width = img_size

### Generate tile

In [10]:
columns = math.floor(ss_width / image_width)
columns

32

In [11]:
all_rows = []
for i in range(len(images)):
    if i % columns == 0:
        all_rows.append([])
    current = all_rows[len(all_rows) - 1]
    current.append(images[i])

In [12]:
len(all_rows)

1563

In [13]:
ranges = []
for i in range(math.ceil(len(all_rows) / columns)):
    start = i * columns
    end = (i + 1) * columns
    if i == math.ceil(len(all_rows) / columns) - 1:
        end = len(all_rows)
    ranged = range(start, end)
    ranges.append(ranged)

In [14]:
len(ranges), columns

(49, 32)

In [15]:
ranges[:5], ranges[-5:]

([range(0, 32), range(32, 64), range(64, 96), range(96, 128), range(128, 160)],
 [range(1408, 1440),
  range(1440, 1472),
  range(1472, 1504),
  range(1504, 1536),
  range(1536, 1563)])

In [16]:
leftover = ss_width % columns
row_padding = []
for i in range(leftover):
    row_padding.append(0)
row_padding

[]

In [17]:
chunk_padding = []
for i in range(2048 - columns * img_size):
    chunk_padding.append([])
    current = chunk_padding[len(chunk_padding) - 1]
    for p in range(ss_width):
        current.append(0)
len(chunk_padding)

0

### Split in multiple tiles

In [18]:
len(all_rows[0])

32

In [19]:
ss_width - len(ranges[0]) * img_size

0

In [20]:
img_size

64

In [21]:
os.makedirs(os.path.join(data_folder, 'tiles'), exist_ok=True)

In [22]:
for range_counter in range(len(ranges)):
    full_combined = []
    for r in ranges[range_counter]:
        row_digits = all_rows[r]
        reshaped = []
        for i in range(len(row_digits)):
            digit = row_digits[i]
#             digit.resize((28,28))
            reshaped.append(digit)
        combined = []
        for i in range(image_height):
            combined.append([])
            current = combined[len(combined) - 1]
            for j in range(len(row_digits)):
                current.extend(reshaped[j][i])
            padding_needed = ss_width - len(row_digits) * image_width
            for p in range(padding_needed):
                current.append(0)
        full_combined.extend(combined)
    row_padding_needed = ss_width - len(full_combined)
    wtf = row_padding_needed
    for rp in range(row_padding_needed):
        current = []
        for p in range(ss_width):
            current.append(0)
        full_combined.append(current)
    test = full_combined
    rgba_combined = []
    for row in full_combined:
        rgba_combined.append([])
        current = rgba_combined[len(rgba_combined) - 1]
        for item in row:
            current.append((item, item, item, 255))
    rgba_combined = np.asarray(rgba_combined)
    rgba_combined = rgba_combined.astype(np.uint8)
    im = Image.fromarray(np.asarray(rgba_combined), mode='RGBA')
    name = f'meteotn_tile_solid_{range_counter}.png'
    name = os.path.join(data_folder, 'tiles',  name)
    im.save(name)

### Save embedding coordinates

In [23]:
umap_train_emb = os.path.join(radar_folder, 'train_emb.npz')
umap_train_emb = np.load(umap_train_emb)['arr_0']

umap_train_emb = umap_train_emb[150000:200000]

umap_train_emb.shape

(50000, 5)

In [24]:
dims = ((0, 2), (0, 1), (1, 2))

for dim0, dim1 in dims:
    outfile = f'umap_dims_{dim0}_{dim1}.json'
    with open(os.path.join(data_folder, outfile), 'w') as outfile:
        json.dump(umap_train_emb[:, (dim0, dim1)].tolist(), outfile)

### Load color ramp

In [26]:
color_ramp = Image.open(os.path.join(data_folder, 'YlGnBu.png'))
color_ramp

In [27]:
with open(os.path.join(data_folder, 'color_ramp.json'), 'w') as outfile:
    json.dump(np.array(color_ramp)[0, :, :-1].tolist(), outfile)

### Calculate Wet Area Ratio

In [28]:
images_war = np.sum(images > 36, axis=(1, 2)) / (64 * 64 * np.pi)
images_war.min(), images_war.max()

(0.0, 0.17306545813752486)

In [29]:
images_war = ((images_war / images_war.max()) * 255).round()

In [30]:
with open(os.path.join(data_folder, 'umap_train_war.json'), 'w') as outfile:
    json.dump(images_war.tolist(), outfile)

### Generate dates for set

In [31]:
image_dates = []

metadata = pd.read_csv('/data1/scidata/meteotn_data/run_metadata.csv', index_col=0)
metadata['date'] = pd.to_datetime(metadata.start_datetime)

for _, i in metadata.iterrows():
    for j in range(i.run_length):
        d = str(i.date + timedelta(minutes=(5 * j)))
        image_dates.append(d)
            
image_dates[280:290]

['2010-11-01 23:30:00',
 '2010-11-01 23:35:00',
 '2010-11-01 23:40:00',
 '2010-11-01 23:45:00',
 '2010-11-01 23:50:00',
 '2010-11-01 23:55:00',
 '2010-11-02 00:00:00',
 '2010-11-02 00:05:00',
 '2010-11-02 00:10:00',
 '2010-11-02 00:15:00']

In [32]:
image_dates = image_dates[150000:200000]
print(len(image_dates))

with open(os.path.join(data_folder, 'umap_train_dates.json'), 'w') as outfile:
    json.dump(image_dates, outfile)

50000
